<a href="https://colab.research.google.com/github/christopherGuan/image_recommander_system/blob/main/GIT_LARGE_TEXTCAPS_multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
!nvidia-smi
import torch
torch.cuda.is_available()
import tensorflow as tf
tf.test.gpu_device_name()

Thu May 18 01:30:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

'/device:GPU:0'

## Build Up Environment

In [2]:
%%capture
!wget 'https://aka.ms/downloadazcopy-v10-linux'
!mv downloadazcopy-v10-linux downloadazcopy-v10-linux.tar.gz
!tar -xf downloadazcopy-v10-linux.tar.gz
%mv azcopy_linux_amd64_10.18.1/azcopy /usr/bin/
!rm downloadazcopy-v10-linux.tar.gz -r azcopy_linux_amd64_10.18.1

In [3]:
%%capture
!pip install git+https://github.com/microsoft/azfuse.git

In [4]:
%%capture
!git clone https://github.com/microsoft/GenerativeImage2Text.git
%cd GenerativeImage2Text

In [5]:
%%capture
!pip install -r requirements.txt

In [6]:
%%capture
!python setup.py build develop

## Generate one caption for one image

In [8]:
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
        'image_path': 'aux_data/images/2.jpg', \
        'model_name': 'GIT_LARGE_TEXTCAPS', \
        'prefix': '', \
  }"

2023-05-18 01:34:25.395419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
a view of a mountain with a forest in the background.


## Generate multiple caption for multiple image

In [11]:
#Multiple captions for multiple images
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_multiple_image', \
        'image_path': 'aux_data/images/', \
        'model_name': 'GIT_LARGE_TEXTCAPS', \
        'prefix': '', \
  }"

2023-05-18 01:46:58.066602: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Unzip image folder

In [ ]:
%cd ..

/content


In [ ]:
!unzip -qq coco_dup_300.zip

replace __MACOSX/._coco_dup_300? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace __MACOSX/coco_dup_300/._000000274659.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace __MACOSX/coco_dup_300/._000000274659.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
!mv coco_dup_300 GenerativeImage2Text/aux_data

In [ ]:
%%capture
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_multiple_image', \
        'image_path': 'aux_data/coco_dup_300/', \
        'model_name': 'GIT_LARGE_TEXTCAPS', \
        'prefix': '', \
  }"

## Another method to generate multiple captions

In [13]:
%%shell

for i in {1..2}
do
   echo $i
   AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
        'image_path': ['aux_data/images/$i.jpg'], \
        'model_name': 'GIT_LARGE_TEXTCAPS', \
        'prefix': '', \
  }"
done > large_textcaps_2.csv

2023-05-18 01:51:12.739355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-18 01:51:45.415174: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#upload 50 prepared images
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()
!unzip -qq images_num.zip
!mv images_num aux_data

Saving images_num.zip to images_num.zip


In [ ]:
%%shell
for i in {51..60}
do
   AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
        'image_path': ['aux_data/images_num/$i.jpg'], \
        'model_name': 'GIT_LARGE_TEXTCAPS', \
        'prefix': '', \
  }"
done > large_textcaps_60.csv

In [ ]:
files.download('large_textcaps_60.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Train Model

use the base model in another file

In [ ]:
!python -m generativeimage2text.train -p "{'type': 'forward_backward_example', \
                'image_files': ['aux_data/images/3.jpg'], \
                'captions': ['Shen Yum Performing Arts logo with a yellow background.'], \
            }"

2023-01-05 23:02:11,062.062 34550:train.py:309   <module>(): param:
{'captions': ['Shen Yum Performing Arts logo with a yellow background.'],
 'image_files': ['aux_data/images/3.jpg'],
 'model_name': 'GIT_LARGE_TEXTCAPS',
 'type': 'forward_backward_example'}
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/content/GenerativeImage2Text/generativeimage2text/train.py", line 312, in <module>
    locals()[function_name](**kwargs)
TypeError: forward_backward_example() got an unexpected keyword argument 'model_name'


In [ ]:
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
      'image_path': 'aux_data/images/mov.jpg', \
      'model_name': 'GIT_BASE', \
      'prefix': '', \
}"

tcmalloc: large alloc 1933754368 bytes == 0x45450000 @  0x7f852b94b1e7 0x4d30a0 0x5dede2 0x61033f 0x5aab9b 0x47c416 0x6170f1 0x4f7916 0x4997a2 0x5d8868 0x4990ca 0x55d078 0x5d8941 0x5d8416 0x55f797 0x55cd91 0x55d743 0x627376 0x5aaeb9 0x4990ca 0x55cd91 0x5d8941 0x4990ca 0x55cd91 0x5d8941 0x5d8506 0x677b11 0x677c54 0x678029 0x7f852b548c87 0x5e1baa
extra large movie poster image


In [ ]:
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
      'image_path': 'aux_data/images/mov.jpg', \
      'model_name': 'GIT_LARGE_TEXTCAPS', \
      'prefix': '', \
}"

a poster for the movie matrix.


In [ ]:
%%shell

while read -r file_name;
do
   echo "$file_name"
done < <(tail -n +2 test_5.csv)

0,51,253ce025df6ab8c7.jpg
1,52,5d2951b34750003f.jpg
2,53,af95cc6e5ce4a30c.jpg
3,54,04f7e547e9b446c6.jpg
4,55,9d1d4f4fbc733656.jpg


## Save for later

In [ ]:
with open('/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py','r') as file:
    data = file.readlines()
print(data[329])

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
!python -m generativeimage2text.data_prepare -p "{'type': 'generate_imagenet_unique_names'}"